## MoSDeF-Signac Tutorial
### Solvated Surface Screening
---
This workshop is presented (WIP) at the 2023 CECAM Tutorials in Mainz Germany
23-09-27
Developed by Cal Craven, Co Quach, Kieran Nehils-Puleo, and Eric Jankowski

### Tutorial summary
This tutorial aims to familiarize a molecular simulations researcher on the ways to 
control a workflow managed by [Signac.](https://docs.signac.io/en/latest/) This is
an important tool for accessing and storing workflow metadata such that the workflow
is standardized and [extensible](https://doi.org/10.1080/00268976.2020.1742938), as 
part of the TRUE nature of computational research.

The following workflows will show how to:
1. Calculate the interfacial structures of different water models. Water models include:
    - SPC
    - SPC/E (in)
    - SPC-flex
    - TIP3P-original (in)
    - TIP3P-modified
    - OPC (in)
2. Build functional workflows that pull specifications from Signac job criteria.
3. Run HOOMD simulations and process data in a standardized format for easy extension of the project aims.

OTHERS Water ForceFields to Include
---
CVFF
BLXL
MG
All the tip4ps
SPC-2site

### Learning Guidelines
1. How to operate/set up a signac workflow
2. Using mBuild recipes/scripts for functional workflows
3. How to pass job metadata to a simulation process
4. HOOMD simulations
5. Data processing

### Changes for individual tinkering/things you can change
1. Which surfaces to build
2. Which water models to choose
3. Which simulation statepoints you run
4. Modifying the plotting code
5. Optional details on how to set up signac-dashboard
6. More details on setting up/checking project status from CLI

### TODOS:
1. warnings </br>
    - mbuild periodic kdtree warnings 
    - Compound.box.lengths < Compound.boundingbox.lengths
    - charges
    - extra links to signac
    - missing improper types
2. Needs GMSO PR 749 xml-conversions
3. OPLSAA forcefield for surface is bulky/ needs doi
4. Cool results analysis
5. Run hoomd sims on multi-cores
6. Time test
7. Add explanation text
8. Question and answer text (fill in response, or code)
9. Slow packmol

## GOOGLE COLAB Setup
---
Run these next three panes only if you are using Google Collab for this notebook

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install_miniforge()

import condacolab
condacolab.check()

!conda install mamba
!git clone https://github.com/mosdef-hub/reproducibility_study.git
!mamba env update -n base reproducibility_study/environment.yml


!mamba env update -n base -f reproducibility_study/environment.yml

In [ ]:
!mamba install mbuild foyer gmso hoomd gsd

In [ ]:
!git clone --branch colab-installation https://github.com/CalCraven/surface_coatings.git
%cd surface_coatings
!pip install -e .
%cd ..
!git clone https://github.com/CalCraven/CECAM-MoSDeF-Workshop.git
%cd CECAM-MoSDeF-Workshop/solvated_surface_workflow/
!pwd

## Import packages
---

In [1]:
# Import Libraries
import itertools
import os

import signac
import mbuild as mb
import gmso
from gmso.parameterization import apply
import gsd
import unyt as u

from surface_coatings.monolayer import Monolayer
from surface_coatings.solvated_monolayer import SolvatedMonolayer
from surface_coatings.surfaces.silica_interface import SilicaInterface
from surface_coatings.chains.alkylsilane import Alkylsilane

/Users/calcraven/miniconda3/envs/gmso-dev2/lib/python3.11/site-packages/mdtraj/formats/__init__.py:6: DeprecationWarning: 'xdrlib' is deprecated and slated for removal in Python 3.13
  from .xtc import XTCTrajectoryFile
/Users/calcraven/miniconda3/envs/gmso-dev2/lib/python3.11/site-packages/lark/utils.py:163: DeprecationWarning: module 'sre_parse' is deprecated
  import sre_parse
/Users/calcraven/miniconda3/envs/gmso-dev2/lib/python3.11/site-packages/lark/utils.py:164: DeprecationWarning: module 'sre_constants' is deprecated
  import sre_constants
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/mbuild/mbuild/recipes/__init__.py:13: DeprecationWarning: SelectableGroups dict interface is deprecated. Use select.
  entry_points = metadata.entry_points()["mbuild.plugins"]
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/Workshops/2023-CECAM-Workshop/git_repos/gmso/gmso/formats/mol2.py:79: UserWarning: The record type indicator Meta is not supported. Skippi

## Initialize Signac Project
---
The first step is to define the project parameters. The following `grid` function is a standard way to complete all combinations of the specified statepoint dictionary (`sp_gridDict`). 

Run the following cell and evaluate the project that is generated.

In [32]:
def grid(gridspec):
    """Yields the Cartesian product of a `dict` of iterables.

    The input ``gridspec`` is a dictionary whose keys correspond to
    parameter names. Each key is associated with an iterable of the
    values that parameter could take on. The result is a sequence of
    dictionaries where each dictionary has one of the unique combinations
    of the parameter values.
    """
    for values in itertools.product(*gridspec.values()):
        yield dict(zip(gridspec.keys(), values))

project = signac.init_project("./solvated_surface_project")
# generate statepoints:
sp_gridDict = {
    "water_model":["OPC3", "spce"], # forcefield to use
    "temperature":[298.15],  # K
    "chain_density":[3], # chains/nm 
    "chain_length":[4], # n_carbons
    "solvent_box_height": [3], #nm
    "seed":[42901423],
}
        
for sp in grid(sp_gridDict):
    print("Initializing job", sp)
    project.open_job(sp).init()
project

Initializing job {'water_model': 'OPC3', 'temperature': 298.15, 'chain_density': 3, 'chain_length': 4, 'solvent_box_height': 3, 'seed': 42901423}
Initializing job {'water_model': 'spce', 'temperature': 298.15, 'chain_density': 3, 'chain_length': 4, 'solvent_box_height': 3, 'seed': 42901423}


,sp.water_model,sp.temperature,sp.chain_density,sp.chain_length,sp.solvent_box_height,sp.seed
f959300be61c911cc8512fd1619077d2,OPC3,298.15,3,4,3,42901423
fdb859acfdae44e289ad701c747452f1,spce,298.15,3,4,3,42901423


The project creation generates a directory on the hard drive with information related to the project defintion. Here are a list of some of the files.

In [35]:
!ls -a solvated_surface_project/

.         ..        .signac   workspace


The workspace is a directory where the individual jobs that were initialized above are stored. These are differentiated by unique hashes generatedfrom each statepoint, and so there's no risk of accidentally overwriting data for a different job.

Let's look at the `signac_statepoint.json` file in one of our jobs.

In [36]:
!cat solvated_surface_project/workspace/f959300be61c911cc8512fd1619077d2/signac_statepoint.json

{"water_model": "OPC3", "temperature": 298.15, "chain_density": 3, "chain_length": 4, "solvent_box_height": 3, "seed": 42901423}

Nice work! You should see something that looks like a python dictionary with all of the information that was passed in the above cells. To access this job metadata, see the following [signac tutorials.](https://docs.signac.io/en/latest/tutorial.html#interacting-with-the-signac-project)

### Challenge Problem: Try to add a new job to the workspace. Validate that the directory is added properly

In [ ]:
project = signac.get_project("./solvated_surface_project/")
jobSP = { #initialize a dictionary statepoint
    "water_model":"spce", # forcefield to use
    "temperature":???,  # K
    "chain_density":???, # chains/nm 
    "chain_length":???, # n_carbons
    "solvent_box_height": 3, #nm
    "seed":42901423,
}
project.??? # open and initialize the jobSP dictionary you made
project # how many jobs are there now?

## Generate structures
---
Next we will use `mBuild` (which you're already an expert in) to build up the individual components of our surface. The key components in this hierarchical structure are:
1. The surface, made from silica. This will be loaded via premade recipe in the [coated_surfaces](https://github.com/daico007/surface_coatings/tree/main) repository.
2. The chains to attach to the surface. These are alkylsilane chains, built using the [Polymer Builder](https://github.com/mosdef-hub/mbuild/blob/main/mbuild/lib/recipes/polymer.py) recipe in `mBuild`.
3. The water atoms, which are simple three site HOH molecules, made from the SMILES string. 

In [42]:
job_compoundDict = {} # dictionary to store job compounds

# load the water
water = mb.load("O", smiles=True)
water.name = "water"
solvent_density = 0.35 * u.g/u.cm**3 # calculate density in g/cm^3
atomic_density = solvent_density / (u.amu.to("g")) / water.mass # convert density to atoms/nm^3
print(f"Atoms/nm^3 is: {atomic_density.to('nm**-3'):.0f}")

for job in project: #iterate over all jobs using a simple for loop on the `project`
    chain = Alkylsilane(chain_length=job.sp.chain_length) #create a chain
    surface = SilicaInterface()
    surface_area = surface.box.Lx * surface.box.Ly
    n_chains=int(job.sp.chain_density*surface_area)
    graft_pattern = mb.Random2DPattern(n_chains, seed=job.sp.seed)
    monolayer = Monolayer(surface, graft_pattern, chain, n_chains=n_chains)
    monolayer.name = "surface"
    n_waters = int(surface_area*job.sp.solvent_box_height*atomic_density.to("nm**-3").value)
    print(f"Adding {n_waters:d} waters to the system.")
    solvated_monolayer = SolvatedMonolayer(monolayer, water, n_waters, job.sp.solvent_box_height)
    job_compoundDict[job] = { # store the saved components for visualization
        "water":water, 
        "chain": chain, 
        "surface":surface, 
        "solvated_monolayer":solvated_monolayer
    }


Atoms/nm^3 is: 12 nm**(-3)
Adding 882 waters to the system.
Adding 882 waters to the system.


Let's visualize the monolayer that was built.

In [75]:
solvated_monolayer.visualize()

#### Challenge Problem: The water was packed at too low of a density.
Try to pack the water at a higher density. Rerun the line
`solvent_density = 0.35 * u.g/u.cm**3 # g/cm^3` 
at a density of 0.99 for a better packed system.
Also, there looks like there's a gap in between the water box and the top of the surface. Fix that by adding


`solvated_monolayer.children[1].translate([0,0,-0.3])`


To move just the water molecules down by 0.3 nm. This should be added after the line


`solvated_monolayer = SolvatedMonolayer(monolayer, water, n_waters, job.sp.solvent_box_height)`

## Apply Forcefields
---
The forcefields are stored on disk as XML files. These define all of the forces needed for the simulation. Multiple forcefields can be applied to different molecules in the `mBuild` compound but be certain that these forcefields are compatible (i.e. same mixing rule, 1-4 scaling, cutoffs)

We can iterate through the jobs, grab the solvated surfaces, load the water forcefield specified in `job.sp.water_model` from the `xmls/` directory in this project, and create our parameterized GMSO `Topology`. 

First though, let's look at the syntax for the apply step. 

In [76]:
from gmso.parameterization import apply
help(apply)

Help on function apply in module gmso.parameterization.parameterize:

apply(top, forcefields, match_ff_by='molecule', identify_connections=False, identify_connected_components=True, use_molecule_info=False, assert_bond_params=True, assert_angle_params=True, assert_dihedral_params=True, assert_improper_params=False, remove_untyped=False, fast_copy=True)
    Set Topology parameter types from GMSO ForceFields.
    
    Parameters
    ----------
    top: gmso.core.topology.Topology, required
        The GMSO topology on which to apply forcefields
    
    forcefields: ForceField or dict, required
        The forcefield to apply. If a dictionary is used the keys are labels that match
        the molecule name (specified as a label of site), and the values are gmso ForceField objects that gets applied
        to the specified molecule.
        Note: if a Topology with no molecule is provided, this option will only take
        a ForceField object. If a dictionary of ForceFields is provided, 

In [77]:
surfaceFF = gmso.ForceField("xmls/oplsaa.xml") # load a general opls all atom forcefield from the directory

for job in project:
    ff_path = os.path.join("xmls", job.sp.water_model+".xml")
    waterFF = gmso.ForceField(ff_path)
    top = job_compoundDict[job]["solvated_monolayer"].to_gmso()
    top.identify_connections() # generate bonds, angles, and dihedrals
    parameterized_top = apply(
        top, forcefields={"surface": surfaceFF, "water":waterFF}, 
        identify_connected_components=False
    )
    job_compoundDict[job]["parameterized_top"] = parameterized_top
                              
parameterized_top   

/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/Workshops/2023-CECAM-Workshop/git_repos/gmso/gmso/core/atom_type.py:198: UserWarning: Masses are assumed to be in units of g/mol
  warnings.warn(UNIT_WARNING_STRING.format("Masses", "g/mol"))
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/Workshops/2023-CECAM-Workshop/git_repos/gmso/gmso/core/atom_type.py:209: UserWarning: Charges are assumed to be in units of elementary charge
  warnings.warn(
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/Workshops/2023-CECAM-Workshop/git_repos/gmso/gmso/core/forcefield.py:528: UserWarning: ImproperType between atoms SI, OG, OG and OG is missing from the ForceField.
  warnings.warn(msg)
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/Workshops/2023-CECAM-Workshop/git_repos/gmso/gmso/core/forcefield.py:528: UserWarning: ImproperType between atoms opls_1002, opls_1001, opls_1001 and opls_1001 is missing from the ForceField.
  warnin

#### Challenge problem: Identify the aspectes of the parameterized topology
What is the bonded equation that is being used from both of our forcefields?

To identify this, we need to interrogate the `parameterized_topology` object. Components are stored in IndexedSets. These can be accessed via list indexing.

What is the number of unique bond types in the topology?

To identify that, use a filter to look for bond_types with a unique pair of atom classes

In [79]:
parameterized_top.bonds[?]???_type.expression # access index 0 and print then access the bond_type.expression
# this can also be done with angles, dihedrals, and impropers

<BondType HarmonicBondPotential,
 expression: 0.5*k*(r - r_eq)**2,
 id: 7544581584, 
 parameters: {'k': unyt_quantity(251040., 'kJ/(mol*nm**2)'), 'r_eq': unyt_quantity(0.165, 'nm')},
member types: ('opls_1002', 'opls_1001')>

#### Answer should look something like this:
0.5𝑘(𝑟−𝑟𝑒𝑞)2

Potential filters allow the user to specify what is `unique` about the type. These give a flexible set of methods to get the unique types in the system. For instance, there may be a set with unique parameters. Maybe you want to look at the set with unique full potential expressions. In this case, we will sort by the unique names of the atom_classes that make up the bonded atoms.

In [87]:
from gmso.core.views import PotentialFilters

pfilter = PotentialFilters.UNIQUE_NAME_CLASS
print(f"There are {len(parameterized_top.bond_types(filter_by=pfilter))} uniquely named bond types")
# try a different filter from the PotentialFilters class

15

## Run Simulations
---
Now let's run our [HOOMD](https://hoomd-blue.readthedocs.io/en/v4.1.0/) simulations for each job.

In [78]:
def run_hoomd(job, gmso_snapshot, gmso_forces, forces_base_units, n_steps=1000):
    """Simulation configuration and runtime parameters."""
    # Uses HOOMD 3
    # Fix bottom of surface
    import hoomd

    temp = job.sp.temperature * u.K
    kT = temp.to_equivalent("kJ/mol", "thermal").value

    cpu = hoomd.device.CPU()
    sim = hoomd.Simulation(device=cpu, seed=1)
    # sim.create_state_from_gsd("top.gsd") # does not work
    sim.create_state_from_snapshot(gmso_snapshot)
    sim.operations.integrator = hoomd.md.Integrator(dt=0.001)
    sim.operations.integrator.forces.extend(
        list(set().union(*gmso_forces.values()))
    )
    #sim.operations.integrator.forces.extend(gmso_forces)

    nvt = hoomd.md.methods.NVT(
        kT=kT, tau=1.0, filter=hoomd.filter.All()
    )
    sim.operations.integrator.methods.append(nvt)

    sim.state.thermalize_particle_momenta(filter=hoomd.filter.All(), kT=kT)
    thermodynamic_properties = hoomd.md.compute.ThermodynamicQuantities(
        filter=hoomd.filter.All()
    )

    sim.operations.computes.append(thermodynamic_properties)
    logger = hoomd.logging.Logger()
    logger.add(thermodynamic_properties)
    import os
    if os.path.exists(job.fn('trajectory.gsd')):
        os.remove(job.fn("trajectory.gsd"))
    gsd_writer = hoomd.write.GSD(
        filename=job.fn('trajectory.gsd'),
        trigger=hoomd.trigger.Periodic(1000),
         mode='xb',
         filter=hoomd.filter.All(),
         logger=logger
    )
    sim.operations.writers.append(gsd_writer)
    outlogger = hoomd.logging.Logger(categories=['scalar', 'string'])
    outlogger.add(sim, quantities=['timestep', 'tps'])
    outlogger.add(thermodynamic_properties, ['kinetic_temperature'])
    table = hoomd.write.Table(
        trigger=hoomd.trigger.Periodic(period=100),
        logger=outlogger
    )
    sim.operations.writers.append(table)
    sim.run(n_steps) 
    



/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/Workshops/2023-CECAM-Workshop/git_repos/gmso/gmso/external/convert_hoomd.py:564: UserWarning: Orthorhombic box detected
  warnings.warn("Orthorhombic box detected")
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/Workshops/2023-CECAM-Workshop/git_repos/gmso/gmso/external/convert_hoomd.py:217: NotYetImplementedWarning: Only writing particle, bond, angle, proper and improper dihedral information.Special pairs are not currently written to GSD files
  warnings.warn(
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/Workshops/2023-CECAM-Workshop/git_repos/gmso/gmso/external/convert_hoomd.py:276: UserWarning: Shifting coordinates to [-L/2, L/2]
  warnings.warn("Shifting coordinates to [-L/2, L/2]")
/Users/calcraven/Dropbox/Mac/Documents/Vanderbilt/Research/MoSDeF/Workshops/2023-CECAM-Workshop/git_repos/gmso/gmso/external/convert_hoomd.py:380: UserWarning: 4142 bonds detected
  warnings.warn(f"

In [ ]:
from gmso.external import to_hoomd_forcefield, to_hoomd_snapshot

base_units = { # this unit sytems lets hoomd non-dimensionalize all parameters in the forcefield
    "mass": u.g / u.mol,
    "length": u.nm,
    "energy": u.kJ / u.mol,
}
for job in project:
    parameterized_top = job_compoundDict[job]["parameterized_top"]
    gmso_snapshot, snapshot_base_units = to_hoomd_snapshot( # TODO: simulation forces and connections are ordered differently
        parameterized_top, base_units=base_units
    )
    gmso_forces, forces_base_units = to_hoomd_forcefield( #TODO: can't handle dimensionless parameters currently, PR incoming
        parameterized_top,
        r_cut=1.4,
        base_units=base_units,
        pppm_kwargs={"resolution": (24,24,24), "order": 5},
    )

    run_hoomd(job, gmso_snapshot, gmso_forces, forces_base_units) # Run the simulation and save data to the workspace direcotires

In [ ]:
!head solvated_surface_project/workspace/f959300be61c911cc8512fd1619077d2/trajectory.gsd

## Analyze/record results
---
TODO: LETS PLOT SOMETHING INTERESTING

In [ ]:
import gsd
data = gsd.hoomd.read_log('trajectory.gsd')
timestep = data['configuration/step']
potential_energy = data[
    'log/md/compute/ThermodynamicQuantities/potential_energy']

fig, ax = plt.subplots(1,1,figsize=(5, 3.09))
ax.plot(timestep, potential_energy)
ax.set_xlabel('timestep')
ax.set_ylabel('potential energy')
fig.show()